<a href="https://colab.research.google.com/github/DannyTheFlower/PDA_Course/blob/main/practices/practice_4/PDA_P4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns
from gensim.utils import simple_preprocess
import numpy as np
import gensim.parsing.preprocessing as pp
from gensim.models import Word2Vec, KeyedVectors
import gensim

from pymystem3 import Mystem # для лемматизации
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


# Постановка задачи (на практику 4) и загрузка данных

В рамках практики по обработке текста поработаем с реальными данными, содержательно близкими к данным с первой практики, - из Конструктора ОП. В этот раз поработаем с описанием дисциплин с помощью языковых моделей.


Задача на практику - предобработать датасет:
1. получить текстовое описание каждой дисциплины датасета
2. выполнить предобработку текстовых данных
3. построить для дисциплины вектор на основе word2vec
4. потестировать текстовые представления

Задач не много, но все затратные по времени.

In [2]:
!gdown 1onuImgIOYhiqwnBuFsDnvepD2R5Gkebi

Downloading...
From: https://drive.google.com/uc?id=1onuImgIOYhiqwnBuFsDnvepD2R5Gkebi
To: /content/df_up_courses_202308.csv
100% 1.88G/1.88G [00:21<00:00, 86.4MB/s]


Из информации о дисциплине у нас есть следующая:
1. course_id - номер дисциплины (может повторяться)
2. course_title - название дисциплины
3. description - краткое описание дисциплины
4. section - разделы дисциплины
5. topic - темы, входящие в раздел дисциплины
6. qualification - уровень образования (бакалавриат, магистратура, специалитет)
7. unit_id - айди факульета
8. up_id - айди учебного плана (образовательной прогарммы)
9. up_title - название учебного плана (образовательной программы)
10. start_year - год набора
11. module_name - название модуля, в который входит дисциплина

In [3]:
df = pd.read_csv("/content/df_up_courses_202308.csv")

In [4]:
df.head()

,Unnamed: 0,course_id,course_title,description,section,topic,qualification,unit_id,up_id,up_title,start_year,module_name
0,0,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,"Понятия ИС, КИС, АСУ, АСУП. ИС как инструмент ...",bachelor,14.0,6905,Программирование и интернет-технологии,2018,"Специализация ""Информационные системы и технол..."
1,1,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,"Понятия ИС, КИС, АСУ, АСУП. ИС как инструмент ...",bachelor,14.0,6938,Программирование и интернет-технологии,2019,"Специализация ""Информационные системы и технол..."
2,2,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,"Понятия ИС, КИС, АСУ, АСУП. ИС как инструмент ...",bachelor,14.0,7014,Программирование и интернет-технологии,2020,"Специализация ""Разработка информационных систе..."
3,54,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,Различные методики управления организацией,bachelor,14.0,6905,Программирование и интернет-технологии,2018,"Специализация ""Информационные системы и технол..."
4,55,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,Различные методики управления организацией,bachelor,14.0,6938,Программирование и интернет-технологии,2019,"Специализация ""Информационные системы и технол..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039709 entries, 0 to 1039708
Data columns (total 12 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Unnamed: 0     1039709 non-null  int64  
 1   course_id      1039709 non-null  int64  
 2   course_title   1039709 non-null  object 
 3   description    1038033 non-null  object 
 4   section        1039709 non-null  object 
 5   topic          1039709 non-null  object 
 6   qualification  1039709 non-null  object 
 7   unit_id        1039709 non-null  float64
 8   up_id          1039709 non-null  int64  
 9   up_title       1039709 non-null  object 
 10  start_year     1039709 non-null  int64  
 11  module_name    1039709 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 95.2+ MB


## Предобработка датасета

Преобразуйте датасет так, чтобы одна строка описывала одну дисциплину в учебном плане. Отдельным столбцом должен храниться весь текст, который описывает дисциплину: description, section, topic. Без дубликатов содержания разделов и тем внутри дисциплины.

Сгруппируем данные таким образом, чтобы объединить текстовые поля для каждой уникальной дисциплины (по course_id и up_id). Исключаем NaN значения и преобразуем все в строки перед объединением. Присоединяем тексты, удаляем дубликаты, разделяем их символом новой строки.

In [22]:
def aggregate_texts(series):
    valid_texts = [str(text) for text in series if pd.notnull(text)]
    return '. '.join(set(valid_texts))

df_aggregated = df.groupby(['course_id', 'up_id']).agg({
    'description': aggregate_texts,
    'section': aggregate_texts,
    'topic': aggregate_texts,
    'course_title': 'first',
    'qualification': 'first',
    'unit_id': 'first',
    'up_title': 'first',
    'start_year': 'first',
    'module_name': 'first'
}).reset_index()

Теперь создадим новый столбец, который объединяет тексты из description, section и topic

In [26]:
df_aggregated['full_description'] = df_aggregated[['course_title', 'description', 'section', 'topic']].apply(lambda x: '. '.join(x), axis=1)

Мы можем удалить старые текстовые столбцы, так как они больше не нужны

In [27]:
df_final = df_aggregated.drop(columns=['description', 'section', 'topic'])

Проверим результат

In [28]:
df_final

,course_id,up_id,course_title,qualification,unit_id,up_title,start_year,module_name,full_description
0,2626,6905,Философия,bachelor,8.0,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Философия. Курс «Философия» даёт целостное пре...
1,2626,6938,Философия,bachelor,8.0,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...
2,2626,6974,Философия,bachelor,8.0,Информатика и программирование,2020,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...
3,2626,7014,Философия,bachelor,8.0,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...
4,2626,7136,Философия,bachelor,8.0,Информатика и программирование,2021,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...
...,...,...,...,...,...,...,...,...,...
63964,25731,7255,Основы планирования научных исследований и упр...,master,63.0,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (RU),Основы планирования научных исследований и упр...
63965,25732,7255,Основы планирования научных исследований и упр...,master,63.0,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (EN),Основы планирования научных исследований и упр...
63966,25773,7236,Дифференциальные уравнения,bachelor,51.0,Фотоника и оптоинформатика,2022,Обязательные дисциплины ОП,Дифференциальные уравнения. Дифференциальные у...
63967,30918,7201,Функциональная безопасность киберфизических си...,master,13.0,Функциональная безопасность беспилотных трансп...,2022,Обязательные дисциплины,Функциональная безопасность киберфизических си...


Преобразования выполняются не слишком быстро, рекомендую сохраниться.

In [29]:
df_final.to_csv('transformed_dataset.csv', index=False)

## Предобработка описания дисциплины

Пора поработать со столбцом с текстовым содержанием дисциплины. Минимально понадобятся следующие шаги предобработки:
1. Токенизация
2. Удаление пунктуации
3. Лемматизация
4. Удаление лишних пробелов внутри текста

Чтобы не слишком углубляться в предобработку, рекомендую воспользоваться реализованными в gensim методами. Примеры можно найти в ноутбуке по ссылке: https://colab.research.google.com/drive/1Lxe0_dPxF5_nNC1hnHTRc01rnMmUVCXa?usp=sharing

In [31]:
pre_df = pd.read_csv('transformed_dataset.csv')

In [32]:
pre_df.shape

(63969, 9)

In [33]:
pre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63969 entries, 0 to 63968
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   course_id         63969 non-null  int64  
 1   up_id             63969 non-null  int64  
 2   course_title      63969 non-null  object 
 3   qualification     63969 non-null  object 
 4   unit_id           63969 non-null  float64
 5   up_title          63969 non-null  object 
 6   start_year        63969 non-null  int64  
 7   module_name       63969 non-null  object 
 8   full_description  63969 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 4.4+ MB


In [ ]:
!pip install pymorphy2

In [35]:
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
import string
from nltk.corpus import stopwords
import pymorphy2

In [36]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [37]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
analyzer = pymorphy2.MorphAnalyzer()

In [38]:
def text_to_wordlist(text, remove_stopwords=True):
    text = re.sub("[^а-яА-ЯёЁa-zA-Z&]"," ", text)

    words = text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("russian") + stopwords.words("english"))
        words = [w for w in words if not w in stops]
    norm_words = [analyzer.parse(word)[0].normal_form for word in words]
    return norm_words

# def text_to_sentences(text, tokenizer, remove_stopwords=False):
#     raw_sentences = tokenizer.tokenize(text.strip())
#     sentences = []
#     for raw_sentence in raw_sentences:
#         if len(raw_sentence) > 0:
#             sentences.append(text_to_wordlist(raw_sentence, remove_stopwords=True))
#     return sentences

In [39]:
# def process_description(description):
#     return text_to_sentences(description, tokenizer, remove_stopwords=True)

pre_df['processed_description'] = pre_df['full_description'].apply(text_to_wordlist)

In [40]:
pre_df

,course_id,up_id,course_title,qualification,unit_id,up_title,start_year,module_name,full_description,processed_description
0,2626,6905,Философия,bachelor,8.0,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный..."
1,2626,6938,Философия,bachelor,8.0,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный..."
2,2626,6974,Философия,bachelor,8.0,Информатика и программирование,2020,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный..."
3,2626,7014,Философия,bachelor,8.0,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный..."
4,2626,7136,Философия,bachelor,8.0,Информатика и программирование,2021,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный..."
...,...,...,...,...,...,...,...,...,...,...
63964,25731,7255,Основы планирования научных исследований и упр...,master,63.0,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (RU),Основы планирования научных исследований и упр...,"[основа, планирование, научный, исследование, ..."
63965,25732,7255,Основы планирования научных исследований и упр...,master,63.0,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (EN),Основы планирования научных исследований и упр...,"[основа, планирование, научный, исследование, ..."
63966,25773,7236,Дифференциальные уравнения,bachelor,51.0,Фотоника и оптоинформатика,2022,Обязательные дисциплины ОП,Дифференциальные уравнения. Дифференциальные у...,"[дифференциальный, уравнение, дифференциальный..."
63967,30918,7201,Функциональная безопасность киберфизических си...,master,13.0,Функциональная безопасность беспилотных трансп...,2022,Обязательные дисциплины,Функциональная безопасность киберфизических си...,"[функциональный, безопасность, киберфизический..."


In [41]:
df_final['full_description'].iloc[0]

'Философия. Курс «Философия» даёт целостное представление о специфике философии как актуальной формы мышления, о месте и роли философствования в различных средах: повседневной, социальной, коммуникационной, научной.Этот курс для тех, кто хочет узнать, к примеру, о том, что осталось в нашем мышлении неизменным со времен античности, и почему наука, какой мы знаем ее сегодня, не появилась бы без Декарта. Философия для тех, кому важно знать, как избавиться от ошибочных суждений, развить критическое мышление и попробовать себя в роли мастера аргументации в режиме и оффлайн, и онлайн.. Соматическая определенность мышления. Структуры мышления. Мышление и предметный мир. Социальная определенность мышления. Телесность и аффективность в мышлении.. Критическое мышление как задача философии, науки, искусства, повседневных практик.. Способы и границы систематизации предметов мышления: логика, категориальные структуры.. Нейронаука о мышлении и сознании человека. Проблемы биополитики.. Антропоцентриз

In [42]:
pre_df['processed_description'].iloc[0]

['философия',
 'курс',
 'философия',
 'давать',
 'целостный',
 'представление',
 'специфика',
 'философия',
 'актуальный',
 'форма',
 'мышление',
 'место',
 'роль',
 'философствование',
 'различный',
 'среда',
 'повседневный',
 'социальный',
 'коммуникационный',
 'научный',
 'курс',
 'тот',
 'хотеть',
 'узнать',
 'пример',
 'остаться',
 'наш',
 'мышление',
 'неизменный',
 'время',
 'античность',
 'почему',
 'наука',
 'знать',
 'сегодня',
 'появиться',
 'декарт',
 'философия',
 'тот',
 'кто',
 'важно',
 'знать',
 'избавиться',
 'ошибочный',
 'суждение',
 'развить',
 'критический',
 'мышление',
 'попробовать',
 'роль',
 'мастер',
 'аргументация',
 'режим',
 'офлайн',
 'онлайн',
 'соматический',
 'определённость',
 'мышление',
 'структура',
 'мышление',
 'мышление',
 'предметный',
 'мир',
 'социальный',
 'определённость',
 'мышление',
 'телесность',
 'аффективность',
 'мышление',
 'критический',
 'мышление',
 'задача',
 'философия',
 'наука',
 'искусство',
 'повседневный',
 'практика',
 '

Не будет лишним в этот момент сохраниться. Лемматизация выполняется сравнительно долго.

In [43]:
pre_df.to_csv('tokenized_dataset.csv', index=False)

Далее обучим модель Word2Vec для получения эмбеддингов описаний дисциплины.

In [46]:
model = Word2Vec(pre_df.processed_description.tolist(), min_count=5, vector_size=300, workers=4, window=10)

Сохраним обученную модель.

In [47]:
model_path = "op_w2v"
model.save(model_path)

Посмотрим полученный вектор.

In [48]:
model.wv.get_vector("обучение")

array([-3.7946947 ,  2.1676989 ,  3.87688   , -4.348506  , -5.18349   ,
        0.355687  ,  5.2747154 , -0.01001083,  2.40639   ,  5.2152076 ,
       -1.3233546 , -1.2802994 ,  5.5626154 ,  1.652657  ,  2.4082713 ,
       -0.15900299,  1.5078481 ,  2.6510668 ,  2.3725922 ,  4.270508  ,
       -2.1649694 ,  0.45628363, -5.777483  , -3.936162  ,  1.1863049 ,
        0.51794255, -2.8545117 ,  3.4434865 ,  0.46622238,  0.3097376 ,
       -5.8205295 , -3.7761686 ,  2.5373182 ,  3.585165  , -0.25817055,
       -1.3679944 , -0.10033846,  1.2846544 ,  2.5329087 ,  3.666508  ,
        8.491078  ,  4.958943  , -5.5734944 ,  1.9952881 ,  2.5807285 ,
       -2.832535  ,  1.1300126 , -2.046128  ,  1.3591429 , -6.7967153 ,
        5.2832737 , -1.0755453 , -0.7362062 ,  1.1835595 ,  2.6954424 ,
        1.4522196 , -3.8856018 ,  3.5800393 ,  3.6305153 , -0.7982123 ,
        1.6611463 , -1.8734745 , -0.32491928,  5.184568  ,  2.3554535 ,
       -2.5271263 ,  0.8033313 ,  3.6005747 ,  4.995247  , -2.17

Посмотрим слова, наиболее близкие к "машинному обучению".

In [49]:
model.wv.most_similar(["обучение", "машинный"])

[('дикторский', 0.4575808048248291),
 ('кластеризация', 0.42500603199005127),
 ('мо', 0.42051422595977783),
 ('нейросетевой', 0.41029277443885803),
 ('регрессия', 0.4046062231063843),
 ('регрессионный', 0.39025482535362244),
 ('предобработка', 0.38576382398605347),
 ('аугментация', 0.38441047072410583),
 ('покрывать', 0.37958067655563354),
 ('сигмоидальный', 0.3691387474536896)]

Вероятно, наиболее тесная связь машинного обучения в описании РПД оказалась именно в описании курса "Распознавание диктора".

In [68]:
pre_df[pre_df["course_title"] == "Распознавание диктора"]

,course_id,up_id,course_title,qualification,unit_id,up_title,start_year,module_name,full_description,processed_description,embedding
15432,11220,7007,Распознавание диктора,master,14.0,Речевые информационные системы,2020,"Специализация ""Речевые информационные системы""",Распознавание диктора. Целью освоения дисципли...,"[распознавание, диктор, цель, освоение, дисцип...","[-0.39349097, -0.5428325, -0.16230409, -0.0648..."
15433,11220,7132,Распознавание диктора,master,14.0,Речевые информационные системы,2021,"Специализация ""Речевые информационные системы""",Распознавание диктора. Целью освоения дисципли...,"[распознавание, диктор, цель, освоение, дисцип...","[-0.39349097, -0.5428325, -0.16230409, -0.0648..."
37661,17958,7187,Распознавание диктора,master,14.0,Речевые технологии и машинное обучение,2022,Технологии обработки звучащей речи,Распознавание диктора. Целью освоения дисципли...,"[распознавание, диктор, цель, освоение, дисцип...","[-0.34792495, -0.544846, -0.20985654, -0.09122..."


Определим функцию, которая возвращает вектор дисциплины как усреднённый вектор слов, встречающихся в описании дисциплины.

In [54]:
def embed(tokens, w2v_model):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    if len(vectors) == 0:
        return np.zeros(w2v_model.vector_size)
    return np.mean(vectors, axis=0)

# Посмотрим такой вектор для первой строки в датафрейме
embed(pre_df['processed_description'].iloc[0], model)

array([ 0.04373798, -0.658152  ,  0.28587788,  0.585062  , -0.00506405,
       -0.9506368 , -0.7301807 , -1.4127291 ,  0.28006163,  0.80679524,
        0.49662745,  0.5839219 ,  0.8435969 , -0.00410467,  0.83937365,
        1.3787162 , -0.10392986,  1.1057804 , -0.4668379 ,  0.28356832,
        0.00726668, -0.11695214,  0.2542862 , -0.6804354 , -0.36416763,
       -0.24713181, -0.00315812, -0.930352  ,  0.3769587 ,  0.5268821 ,
       -0.5061884 , -0.35487095, -1.2084228 , -0.16638039, -0.6477587 ,
        0.14621519,  0.5879475 , -0.49410173, -0.5213617 ,  0.22338594,
        0.40338397,  0.6205956 ,  0.6386228 ,  1.2278776 , -0.00434479,
       -0.67840296, -0.40437394,  0.4405827 , -0.46311954,  0.20607173,
       -0.32967064,  0.41020256,  0.02331948, -0.14914212, -0.48190594,
       -0.8072666 ,  1.0778548 ,  0.3184639 ,  0.51960343,  0.4073525 ,
        0.00586586,  0.5486915 ,  0.55266136, -0.2862532 , -0.15096794,
       -0.43664867, -0.3384156 ,  0.8941135 , -0.3541678 ,  0.53

In [55]:
pre_df["embedding"] = pre_df['processed_description'].apply(lambda x: embed(x, model))
pre_df

,course_id,up_id,course_title,qualification,unit_id,up_title,start_year,module_name,full_description,processed_description,embedding
0,2626,6905,Философия,bachelor,8.0,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный...","[0.043737978, -0.658152, 0.28587788, 0.585062,..."
1,2626,6938,Философия,bachelor,8.0,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный...","[0.043737978, -0.658152, 0.28587788, 0.585062,..."
2,2626,6974,Философия,bachelor,8.0,Информатика и программирование,2020,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный...","[0.043737978, -0.658152, 0.28587788, 0.585062,..."
3,2626,7014,Философия,bachelor,8.0,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный...","[0.043737978, -0.658152, 0.28587788, 0.585062,..."
4,2626,7136,Философия,bachelor,8.0,Информатика и программирование,2021,Модуль обязательных дисциплин,Философия. Курс «Философия» даёт целостное пре...,"[философия, курс, философия, давать, целостный...","[0.043737978, -0.658152, 0.28587788, 0.585062,..."
...,...,...,...,...,...,...,...,...,...,...,...
63964,25731,7255,Основы планирования научных исследований и упр...,master,63.0,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (RU),Основы планирования научных исследований и упр...,"[основа, планирование, научный, исследование, ...","[-0.13769999, 0.2770018, -0.1322378, 0.1976504..."
63965,25732,7255,Основы планирования научных исследований и упр...,master,63.0,Физика и технология наноструктур / Physics and...,2022,Обязательные дисциплины (EN),Основы планирования научных исследований и упр...,"[основа, планирование, научный, исследование, ...","[0.885515, -0.7378298, -0.1361481, 0.5592815, ..."
63966,25773,7236,Дифференциальные уравнения,bachelor,51.0,Фотоника и оптоинформатика,2022,Обязательные дисциплины ОП,Дифференциальные уравнения. Дифференциальные у...,"[дифференциальный, уравнение, дифференциальный...","[-0.8771806, -0.85055345, -0.4685197, 0.018825..."
63967,30918,7201,Функциональная безопасность киберфизических си...,master,13.0,Функциональная безопасность беспилотных трансп...,2022,Обязательные дисциплины,Функциональная безопасность киберфизических си...,"[функциональный, безопасность, киберфизический...","[0.026472429, -0.41985753, -0.2611326, 0.39716..."


Сохраним финальную версию датафрейма.

In [69]:
pre_df.to_csv("embed_dataset.csv", index=False)